Server role membership  

Source code: [https://docs.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-server-role-members-transact-sql?view=sql-server-ver15](https://docs.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-server-role-members-transact-sql?view=sql-server-ver15)

In [1]:
SELECT	roles.principal_id							AS RolePrincipalID
	,	roles.name									AS RolePrincipalName
	,	server_role_members.member_principal_id		AS MemberPrincipalID
	,	members.name								AS MemberPrincipalName
FROM sys.server_role_members AS server_role_members
INNER JOIN sys.server_principals AS roles
    ON server_role_members.role_principal_id = roles.principal_id
INNER JOIN sys.server_principals AS members 
    ON server_role_members.member_principal_id = members.principal_id  
;

(6 rows affected)

Total execution time: 00:00:00.010

RolePrincipalID,RolePrincipalName,MemberPrincipalID,MemberPrincipalName
3,sysadmin,1,sa
3,sysadmin,259,DESKTOP-SV5TDG6\deept
3,sysadmin,260,NT SERVICE\SQLWriter
3,sysadmin,261,NT SERVICE\Winmgmt
3,sysadmin,262,NT Service\MSSQLSERVER
3,sysadmin,264,NT SERVICE\SQLSERVERAGENT


Orpahn user list

Source code: http://sql-articles.com/scripts/identify-orphan-users-in-all-the-databases/

In [2]:
DECLARE @OrphanUser TABLE( DatabaseName VARCHAR(255),
                           UserName     VARCHAR(255),
                           DropScript   VARCHAR(1024));
DECLARE @SQL VARCHAR(MAX);
IF( SELECT SUBSTRING(CONVERT( VARCHAR(25), SERVERPROPERTY('ProductVersion')), 1, CHARINDEX('.', CONVERT(VARCHAR(25), SERVERPROPERTY('ProductVersion'))) - 1)) < 12
    BEGIN
        SET @SQL = 'SELECT ''?'' [DatabaseName],a.name,''USE [?]; DROP LOGIN [''+a.name+''];''
FROM ?.sys.database_principals a
LEFT OUTER JOIN sys.server_principals  b on a.sid=b.sid
WHERE a.type = ''S'' 
AND a.name NOT IN (''dbo'',''sys'',''INFORMATION_SCHEMA'',''guest'') AND a.name NOT LIKE ''%##%'' AND DB_ID(''?'') > 4
AND b.sid IS NULL';
    END;
ELSE
    BEGIN
        SET @SQL = 'SELECT ''?'' [DatabaseName],a.name,''USE [?]; DROP LOGIN [''+a.name+''];''
FROM ?.sys.database_principals a
LEFT OUTER JOIN sys.server_principals  b on a.sid=b.sid
WHERE a.type = ''S'' 
AND a.name NOT IN (''dbo'',''sys'',''INFORMATION_SCHEMA'',''guest'') AND a.name NOT LIKE ''%##%'' AND DB_ID(''?'') > 4
AND b.sid IS NULL AND a.authentication_type=1';
    END;
INSERT INTO @OrphanUser
EXEC sp_MSforeachdb
     @SQL;
SELECT *
FROM @OrphanUser
ORDER BY DatabaseName;

(0 rows affected)

(0 rows affected)

Total execution time: 00:00:00.160

DatabaseName,UserName,DropScript


Database level information

In [6]:
SELECT name,compatibility_level,recovery_model_desc,state_desc  FROM sys.databases

(10 rows affected)

Total execution time: 00:00:00.027

name,compatibility_level,recovery_model_desc,state_desc
master,150,SIMPLE,ONLINE
tempdb,150,SIMPLE,ONLINE
model,150,FULL,ONLINE
msdb,150,SIMPLE,ONLINE
AdventureWorks,150,SIMPLE,ONLINE
AdventureWorks2012,150,SIMPLE,ONLINE
WideWorldImportersDW,130,SIMPLE,ONLINE
test,150,FULL,ONLINE
COVID,150,FULL,ONLINE
Notebook_results,150,FULL,ONLINE


Login server role permissions

In [7]:
SELECT name,CASE WHEN isntgroup =1 AND isntuser=0 THEN 'Windows Group'
    WHEN isntgroup =0 AND isntuser=1 THEN 'Windows Login'
    ELSE 'SQL Login' END AS 'Login Type',dbname,
    CASE WHEN sysadmin = 1 THEN 'sysadmin'
          WHEN securityadmin=1 THEN 'securityadmin'
          WHEN serveradmin=1 THEN 'serveradmin'
          WHEN setupadmin=1 THEN 'setupadmin'
          WHEN processadmin=1 THEN 'processadmin'
          WHEN diskadmin=1 THEN 'diskadmin'
          WHEN dbcreator=1 THEN 'dbcreator'
          WHEN bulkadmin=1 THEN 'bulkadmin'
          ELSE 'Public' END AS 'ServerRole' FROM master.dbo.syslogins

(17 rows affected)

Total execution time: 00:00:00.029

name,Login Type,dbname,ServerRole
sa,SQL Login,master,sysadmin
##MS_SQLResourceSigningCertificate##,SQL Login,master,Public
##MS_SQLReplicationSigningCertificate##,SQL Login,master,Public
##MS_SQLAuthenticatorCertificate##,SQL Login,master,Public
##MS_PolicySigningCertificate##,SQL Login,master,Public
##MS_SmoExtendedSigningCertificate##,SQL Login,master,Public
##MS_PolicyEventProcessingLogin##,SQL Login,master,Public
##MS_PolicyTsqlExecutionLogin##,SQL Login,master,Public
##MS_AgentSigningCertificate##,SQL Login,master,Public
DESKTOP-SV5TDG6\deept,Windows Login,master,sysadmin
